#### Suspend functions
These are special functions which can call other suspend functions. They internally accept continuation as a parameter, while calling other suspending functions, they passe their own continuation. If these functions get suspended, the calling thread is freed. The suspended flow is resumed when "resume" methods are called on the continuation passed by this function.

https://kt.academy/article/cc-under-the-hood

In [1]:
import java.util.*
import java.util.concurrent.CountDownLatch
import kotlin.coroutines.*
import kotlin.concurrent.thread



suspend fun susfun(){
    println("Inside susfun")
}

// susfun() : we cannot directly call a suspending function from regular function or lambda
 



#### suspend functions can be called from other suspend functions

In [2]:
suspend fun susfunCaller(){
    susfun()
}

#### suspend functions can be also be called from suspend lambdas

In [3]:
// { susfun() } : cannot be called from a regular lambda

In [4]:
val suspendingLambda = suspend{ susfun() }

In [5]:
// suspendingLambda() : again cannot be called from regular function

#### Following assignments are equivalent

In [6]:
val suspendingLambda1 = suspend{ susfun()} // Type inferred from the RHS 
val suspendingLambda2 : suspend()->Unit = { susfun()} // Type explicitly declared as a suspending lambda



#### Calling suspending lambda with coroutine using startCoroutine

Note that if the suspending function suspends, the calling thread is freed, it continues executing the next code in the calling function. When the suspending function is resumed, next code (after the point of suspension) in suspending function is executed. We can consider the lambda itself as a suspending function and pass it our custom continuation (as in below code), once the lambda completes execution, resumeWith is called on our custom continuation.

In [7]:


suspendingLambda.startCoroutine(object : Continuation<Unit>{
    
           override val context: CoroutineContext
               get() = EmptyCoroutineContext

           override fun resumeWith(result: Result<Unit>) {
               println("resumed") // This is called when suspendingLambda is resumed 
           }

       })



Inside susfun
resumed


#### A non suspending function capable of running suspending code (as suspending lambda)

In [8]:
fun launch(suspendingLambda : suspend()->Unit){
    suspendingLambda.startCoroutine(object : Continuation<Unit>{
    
           override val context: CoroutineContext
               get() = EmptyCoroutineContext

           override fun resumeWith(result: Result<Unit>) {
               // Don't do anything
           }

       })
}

In [9]:
launch {
    susfun()
}

Inside susfun


#### Suspend and resume with suspendCoroutine

suspendCoroutine executes the lambda providing it the continuation. After lambda execution is complete, the continuation is checked 
to have been called with result (or exception), if it is, the result is returned to the caller as is, else the calling method
is suspended. This suspended method can be resumed later when continuation.resume is called.

In [10]:
suspend fun immediateResume() {

    println("Before")

    // After this call, the lambda passed is called and then immediateResume method is tried to be suspended.
    suspendCoroutine<Unit> { continuation ->
        println("Resuming")
        continuation.resume(Unit) // immediateResume method is not suspended yet, because lambda is still running, but we called resume
        println("will resume")
    }
    // immediateResume method will not suspend because resume was already called in the lambda (and the result (Unit) is already 
    // available in the continuation after execution of the lambda),
    // it will just continue.
    println("After")
}

launch{ immediateResume() }

Before
Resuming
will resume
After


In [11]:
suspend fun immediateResumeWithValue() {

    println("Before")

    // After this call, the lambda passed is called and then the immediateResumeWithValue method is tried to be suspended.
    val result = suspendCoroutine<String> { continuation ->
        println("Resuming")
        continuation.resume("Hello") // The immediateResumeWithValue method is not suspended yet, because lambda is still running, but we called resume
        println("will resume")
    }
     // immediateResumeWithValue method will not suspend because resume was already called in the lambda (and the result (Unit) is already 
    // available in the continuation after execution of the lambda),
    // it will just continue, because it can continue with the result.
    println("result $result")
}

launch { immediateResumeWithValue() } 

Before
Resuming
will resume
result Hello


In [12]:
suspend fun resumeEarlyNoSuspend() {

    println("Before in thread: ${Thread.currentThread().name}")

    suspendCoroutine<Unit> { continuation ->
        thread(name="ResumingThread") {
            println("Starting thread: ${Thread.currentThread().name}")

            // call resume before resumeEarlyNoSuspend method suspends
            continuation.resume(Unit)
            println("Called resume")
            println("At the end of thread: ${Thread.currentThread().name}")
        }

        // Don't give resumeEarlyNoSuspend method a chance to suspend
        Thread.sleep(2000)
    }

    // This code will run in calling thread itself, because suspend didn't happen
    println("Continued in thread: ${Thread.currentThread().name}")

}

launch{ resumeEarlyNoSuspend() } 

Before in thread: Thread-49
Starting thread: ResumingThread
Called resume
At the end of thread: ResumingThread
Continued in thread: Thread-49


In [13]:
suspend fun resumeInAnotherThread() {

    println("Before in thread: ${Thread.currentThread().name}")

    suspendCoroutine<Unit> { continuation ->
        thread(name="ResumingThread") {
            println("Starting thread: ${Thread.currentThread().name}")

            Thread.sleep(2000) // Give main method a chance to suspend
            // Now resume the suspended flow
            continuation.resume(Unit)
            // This will be printed after completion of suspended flow
            println("At the end of thread: ${Thread.currentThread().name}")
        }
    }

    // All code after suspension will run in the resuming thread.
    println("Resumed in thread: ${Thread.currentThread().name}")

}

launch { resumeInAnotherThread() } 
Thread.sleep(3000)

Before in thread: Thread-54
Starting thread: ResumingThread
Resumed in thread: ResumingThread
At the end of thread: ResumingThread


In [14]:
suspend fun resumeInAnotherThreadWithValue() {

    println("Before in thread: ${Thread.currentThread().name}")

    val msg = suspendCoroutine<String> { continuation ->
        thread(name="ResumingThread") {
            println("Starting thread: ${Thread.currentThread().name}")

            Thread.sleep(2000) // Give main method a chance to suspend
            // Now resume the suspended flow
            continuation.resume("Hello world")
            // This will be printed after completion of suspended flow
            println("At the end of thread: ${Thread.currentThread().name}")
        }
    }

    // All code after suspension will run in the resuming thread.
    println("Resumed in thread: ${Thread.currentThread().name}, got msg $msg")

}

launch{ resumeInAnotherThreadWithValue() }
Thread.sleep(3000)

Before in thread: Thread-59
Starting thread: ResumingThread
Resumed in thread: ResumingThread, got msg Hello world
At the end of thread: ResumingThread


#### suspending delay function: while delay is delaying, main thread can continue

In [15]:
suspend fun delay(millis : Long){

    suspendCoroutine<Unit> {
        thread {
            Thread.sleep(millis) // best practice is to use a common SchedulingThreadPoolExecutor than creating a thread each time
            it.resume(Unit)
        }

    }
    
}

#### launch coroutine builder

In [16]:
launch {
    delay(1000)
    println("${Date()} first complete") 
} // Code doesn't wait for resumeInAnotherThread() to complete
launch {
    delay(1000)
    println("${Date()} second complete") 
} // Resume happens in another threads
launch {
    delay(1000)
    println("${Date()} third complete") 
}
Thread.sleep(2000)

Tue Feb 08 22:47:01 IST 2022 first complete
Tue Feb 08 22:47:01 IST 2022 second complete
Tue Feb 08 22:47:01 IST 2022 third complete


##### Note above that all launches were started almost simultaneously because suspending nature of launch, you can see all completing almost simultaneously and even the order is not deterministic.

#### runBlocking coroutine builder

In [17]:
fun runBlocking(code:suspend()->Unit){

    val cdl = CountDownLatch(1)

    code.startCoroutine(object: Continuation<Unit>{
        override val context: CoroutineContext
            get() = EmptyCoroutineContext

        override fun resumeWith(result: Result<Unit>) {
            cdl.countDown()
        }

    })

    cdl.await()
}

In [18]:
runBlocking {
    delay(1000)
    println("${Date()} first complete") 
} // Code doesn't wait for resumeInAnotherThread() to complete
runBlocking {
    delay(1000)
    println("${Date()} second complete") 
} // Resume happens in another threads
runBlocking {
    delay(1000)
    println("${Date()} third complete") 
}
Thread.sleep(4000)

Tue Feb 08 22:47:03 IST 2022 first complete
Tue Feb 08 22:47:04 IST 2022 second complete
Tue Feb 08 22:47:05 IST 2022 third complete


##### Note above that all launches were started one after another due to blocking nature of runBlocking, you can see all completing one after another and the order is maintained.